In [ ]:
"""
Standalone test-suite for the CX and T conjugation rules
defined via QuantumGate.register.  Run this cell once;
it prints any mismatches (or 'All tests passed.').

Requirements
------------
* The classes `PauliPath` and `QuantumGate` plus the
  rules `_cx_rule` and `_t_rule` must already be in scope.
* Uses only numpy + Qiskit, no external test frameworks.

The checks:

CX gate
-------
• All 16 Pauli strings on 2 qubits, for both CX(0→1) and CX(1→0).
• Compares matrix from `_cx_rule` with exact U† P U.

T gate
------
• Single-qubit test on {I,X,Y,Z}.  
• 100 random 4-qubit Pauli strings, apply T on a random qubit.
"""

import itertools, random, numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, Operator

# ------------------------------------------------------------------
# helper functions
# ------------------------------------------------------------------
def pauli_matrix(label: str) -> np.ndarray:
    return Pauli(label).to_matrix()

def cx_matrix(ctrl: int, tgt: int) -> np.ndarray:
    qc = QuantumCircuit(2)
    qc.cx(ctrl, tgt)
    return Operator(qc).data

def sum_paths(paths):
    """Return Σ coeff · matrix for a list[PauliPath]."""
    return sum(p.coeff * p.pauli.to_matrix() for p in paths)

# ------------------------------------------------------------------
# collect failures here
# ------------------------------------------------------------------
failures = []

# ================================================================
# 1) CX exhaustive on 2-qubit Paulis
# ================================================================
labels_2q = ["".join(p) for p in itertools.product("IXYZ", repeat=2)]
for ctrl, tgt in [(0, 1), (1, 0)]:
    U = cx_matrix(ctrl, tgt)
    for lab in labels_2q:
        inp      = PauliPath(1.0, Pauli(lab))
        out      = QuantumGate.get("cx")(inp, ctrl, tgt)
        mat_rule = sum_paths(out)
        mat_ref  = U.conj().T @ pauli_matrix(lab) @ U
        if not np.allclose(mat_rule, mat_ref, atol=1e-12, rtol=1e-12):
            failures.append(("CX", f"ctrl={ctrl}", f"tgt={tgt}", lab))

# ================================================================
# 2) T on single-qubit {I,X,Y,Z}
# ================================================================
T = np.diag([1.0, np.exp(1j * np.pi / 4)])
for lab in "IXYZ":
    inp      = PauliPath(1.0, Pauli(lab))
    out      = QuantumGate.get("t")(inp, 0)
    mat_rule = sum_paths(out)
    mat_ref  = T.conj().T @ pauli_matrix(lab) @ T
    if not np.allclose(mat_rule, mat_ref, atol=1e-12, rtol=1e-12):
        failures.append(("T-single", lab))

# ================================================================
# 3) T embedded in random 4-qubit strings
# ================================================================
for _ in range(100):
    label4 = "".join(random.choice("IXYZ") for _ in range(4))
    q      = random.randrange(4)          # qubit on which T acts

    inp      = PauliPath(1.0, Pauli(label4))
    out      = QuantumGate.get("t")(inp, q)
    mat_rule = sum_paths(out)

    # exact 4-qubit reference
    mats = []
    for i, ch in enumerate(reversed(label4)):  # little-endian
        if i == q:
            mats.append(T.conj().T @ pauli_matrix(ch) @ T)
        else:
            mats.append(pauli_matrix(ch))
    mat_ref = mats[0]
    for m in mats[1:]:
        mat_ref = np.kron(m, mat_ref)

    if not np.allclose(mat_rule, mat_ref, atol=1e-12, rtol=1e-12):
        failures.append(("T-embed", label4, f"q={q}"))

# ------------------------------------------------------------------
# print summary
# ------------------------------------------------------------------
if failures:
    print("FAILURES detected:")
    for item in failures:
        print("  ", item)
else:
    print("All tests passed ✅")


In [ ]:
import itertools, random, numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, Operator

# ---------------------------------------------------------
# helpers
# ---------------------------------------------------------
I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)
PMAT = {"I": I, "X": X, "Y": Y, "Z": Z}

def kron_label(lbl: str) -> np.ndarray:
    """Kronecker product, left-char = qubit-0 (Qiskit big-endian)."""
    mat = PMAT[lbl[0]]
    for ch in lbl[1:]:
        mat = np.kron(mat, PMAT[ch])
    return mat

def sum_paths(paths):
    return sum(p.coeff * p.pauli.to_matrix() for p in paths)

def cnot_matrix(ctrl, tgt):
    qc = QuantumCircuit(2)
    qc.cx(ctrl, tgt)
    return Operator(qc).data

# ---------------------------------------------------------
# actual test
# ---------------------------------------------------------
def run_checks():
    failures = []

    # ---------- CX exhaustive ----------
    labels = ["".join(p) for p in itertools.product("IXYZ", repeat=2)]
    for ctrl, tgt in [(0, 1), (1, 0)]:
        U = cnot_matrix(ctrl, tgt)
        for lab in labels:
            inp = PauliPath(1.0, Pauli(lab))
            out = QuantumGate.get("cx")(inp, ctrl, tgt)
            mat_rule = sum_paths(out)
            mat_ref  = U.conj().T @ Pauli(lab).to_matrix() @ U
            if not np.allclose(mat_rule, mat_ref, atol=1e-12, rtol=1e-12):
                failures.append(("CX", f"ctrl={ctrl}", f"tgt={tgt}", lab))

    # ---------- single-qubit T ----------
    Tmat = np.diag([1.0, np.exp(1j*np.pi/4)])
    for lab in "IXYZ":
        inp = PauliPath(1.0, Pauli(lab))
        out = QuantumGate.get("t")(inp, 0)
        mat_rule = sum_paths(out)
        mat_ref  = Tmat.conj().T @ Pauli(lab).to_matrix() @ Tmat
        if not np.allclose(mat_rule, mat_ref, atol=1e-12, rtol=1e-12):
            failures.append(("T-single", lab))

    # ---------- embedded T (random) ----------
    for _ in range(100):
        lbl = "".join(random.choice("IXYZ") for _ in range(4))
        q   = random.randrange(4)
        inp = PauliPath(1.0, Pauli(lbl))
        out = QuantumGate.get("t")(inp, q)
        mat_rule = sum_paths(out)

        kron_parts = []
        for idx, ch in enumerate(reversed(lbl)):  # little-endian → match to_matrix()
            if idx == q:
                kron_parts.append(Tmat.conj().T @ PMAT[ch] @ Tmat)
            else:
                kron_parts.append(PMAT[ch])
        mat_ref = kron_parts[0]
        for m in kron_parts[1:]:
            mat_ref = np.kron(m, mat_ref)

        if not np.allclose(mat_rule, mat_ref, atol=1e-12, rtol=1e-12):
            failures.append(("T-embed", lbl, f"q={q}"))

    # ---------- summary ----------
    if failures:
        print("FAILURES detected:")
        for item in failures:
            print("  ", item)
    else:
        print("All tests passed ✅")

# run immediately
run_checks()


In [ ]:
# Randomised consistency check: PauliPropagator vs. full state-vector




# ------------------------------------------------------------
# helper generators
# ------------------------------------------------------------

def random_state_label(n: int) -> str:
    """Return a random product state label of length n."""
    symbols = "01+-rl"          # Qiskit’s allowed single-qubit symbols
    return "".join(random.choice(symbols) for _ in range(n))


def random_pauli_label(n: int) -> str:
    """Return a random n-qubit Pauli label (not all-I)."""
    symbols = "IXYZ"
    label = "".join(random.choice(symbols) for _ in range(n))
    if set(label) == {"I"}:                      # ensure non-trivial
        pos = random.randrange(n)
        label = label[:pos] + random.choice("XYZ") + label[pos + 1 :]
    return label


def build_random_circuit(n: int, gate_count: int) -> QuantumCircuit:
    """Construct a random circuit on n qubits with `gate_count` gates (T and CX only)."""
    qc = QuantumCircuit(n, name=f"rand_{n}q_{gate_count}g")
    for _ in range(gate_count):
        if random.choice(("t", "cx")) == "t":
            q = random.randrange(n)
            qc.t(q)
        else:
            ctrl, tgt = random.sample(range(n), 2)
            qc.cx(ctrl, tgt)
    return qc

# random.seed(42)      
     
TOL = 1e-10
failures = []                  

for i in range(1, 200):         # 50 
    n          = random.randint(3, 15)     # qubit 
    gate_count = random.randint(3, 15)     # 
    qc         = build_random_circuit(n, gate_count)

    state_lbl  = random_state_label(n)     #  product-state
    pauli_lbl  = random_pauli_label(n)     # Pauli
    pauli_obs  = Pauli(pauli_lbl)

    # --- Pauli-propagation ---
    prop        = PauliPropagator(qc)
    layers      = prop.propagate(PauliPath(1.0, pauli_obs))   # 
    prop_expect = prop.expectation_pauli_sum(layers[-1], state_lbl)

    # --- state-vector  ---
    sv_expect   = Statevector.from_label(state_lbl) \
                    .evolve(qc) \
                    .expectation_value(pauli_obs).real

    diff   = abs(prop_expect - sv_expect)
    status = "PASS" if diff < TOL else "FAIL"

    # print(f"Trial {i:02d}: qubits={n:2d}, gates={gate_count:2d}, "
    #       f"state='{state_lbl}', observable={pauli_lbl:>{n}}, "
    #       f"prop={prop_expect:+.12f}, sv={sv_expect:+.12f}, "
    #       f"diff={diff:.3e} → {status}")

    if status == "FAIL":
        failures.append((i, qc, state_lbl, pauli_lbl,
                         prop_expect, sv_expect, diff))

print("\n=== Summary ===")
if failures:
    print(f"{len(failures)} failure(s) detected:")
    for fid, qc, s_lbl, p_lbl, p_exp, sv_exp, d in failures:
        print(f"\n--- Failure {fid} ---")
        print(f"state      : '{s_lbl}'")
        print(f"observable : {p_lbl}")
        print(f"propagate  : {p_exp:+.12f}")
        print(f"statevector: {sv_exp:+.12f}")
        print(f"diff       : {d:.3e}")
        print("\nCircuit:")
        print(qc.draw('text'))
else:
    print("All tests passed 🎉")

